# Step

##  Task

## Task

In [4]:
import nltk

In [8]:
stemmer = stem.snowball.SnowballStemmer('english')
tokens = ['truth', 'truthful', 'truthfulness', 'countertruth', 'untruthful', 'truthology']
# Perform stemming on the tokenized words 
tokens_stemmed = [stemmer.stem(x) for x in tokens]
tokens_stemmed

In [12]:
tokens = ['flaw', 'flaws', 'flawed', 'flawless', 'flawlessness', 'flawlessly']
tokens_stemmed = [stemmer.stem(x) for x in tokens]
tokens_stemmed

['flaw', 'flaw', 'flaw', 'flawless', 'flawless', 'flawless']

In [15]:
stemmer = stem.snowball.SnowballStemmer('Russian')
tokens = ['лес', 'лесной', 'лесник', 'лесничий', 'лесничество', 'пролесье']
tokens_stemmed = [stemmer.stem(x) for x in tokens]
tokens_stemmed

['лес', 'лесн', 'лесник', 'леснич', 'лесничеств', 'пролес']

In [16]:
tokens = ['окно', 'окошко', 'подоконник', 'оконный', 'окнище']
tokens_stemmed = [stemmer.stem(x) for x in tokens]
tokens_stemmed

['окн', 'окошк', 'подоконник', 'окон', 'окнищ']

In [ ]:
окн - корінь
забрано закінчення -о та -е
Потрібно розділяти поняття stem(основа слова) та root(корінь)

## Task

## Task

## Task

## Task

In [41]:
from nltk.corpus import wordnet as wn
word='bullet'
syn_list = wn.synsets(word)
for num, i in enumerate(syn_list):
    print(num, i.definition())
choose_syn = syn_list[0]
print('Hypernyms:', choose_syn.hypernyms())
print('Hyponyms:', choose_syn.hyponyms() )
print('meronyms:', choose_syn.member_meronyms())
print('holonyms:', choose_syn.member_holonyms() )

0 a projectile that is fired from a gun
1 a high-speed passenger train
2 (baseball) a pitch thrown with maximum velocity
Hypernyms: [Synset('projectile.n.01')]
Hyponyms: [Synset('dumdum.n.01'), Synset('full_metal_jacket.n.01'), Synset('rifle_ball.n.01'), Synset('rubber_bullet.n.01')]
meronyms: []
holonyms: []


# Step

## Task Форматування

In [39]:
import spacy
from spacy.symbols import NOUN, PROPN, PRON, VERB, ADJ, ADV, ADP
# tag- IN, pos - ADP, conjunction, subordinating or preposition
nlp = spacy.load('en')
# іменники, займенники, дієслова, прикметники, прислівники та підрядні сполучники
target_pos_tags = ('NOUN', 'PROPN', 'PRON', 'VERB', 'ADJ', 'ADV', 'ADP')

In [227]:
def read_lines_fron_txt(file_name):
    
    f = open(file_name)
    f_lines = f.readlines()
    file_in.close()
    return f_lines 

def capitalize_1(word):
    
    if word == word.upper():
        return word
    else:
        return word.capitalize()

In [235]:
def style_headlines(file_in, file_out):
    
    f_lines = read_lines_fron_txt(file_in)
    # append data to file
    f_out = open( file_out, "a+")
    for line in f_lines:
#         print(line)
        doc = nlp(line)
        new_line = capitalize_1(doc[0].text)
        for token in doc[1:-1]:
            curr_token = token.text
            if(token.pos_ in target_pos_tags and token.dep_ != 'prep'):
                curr_token = (capitalize_1(token.text))
            new_line = new_line +  " "*(token.idx - len(new_line)) + curr_token
        new_line = new_line + capitalize_1(doc[-1].text)
#         print(new_line)
        f_out.write(new_line)
    f_out.close()
    

In [236]:
style_headlines(file_in ='examiner-headlines.txt', file_out="examiner-headlines_out.txt")

## Task Вірусні новини

Чи є в заголовку іменовані стуності?
Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
Чи є заголовок позитивно чи негативно забарвлений?

Possible entities:
PERSON	People, including fictional.
NORP	Nationalities or religious or political groups.
FAC	Buildings, airports, highways, bridges, etc.
ORG	Companies, agencies, institutions, etc.
GPE	Countries, cities, states.
LOC	Non-GPE locations, mountain ranges, bodies of water.
PRODUCT	Objects, vehicles, foods, etc. (Not services.)
EVENT	Named hurricanes, battles, wars, sports events, etc.
WORK_OF_ART	Titles of books, songs, etc.
LAW	Named documents made into laws.
LANGUAGE	Any named language.
DATE	Absolute or relative dates or periods.
TIME	Times smaller than a day.
PERCENT	Percentage, including "%".
MONEY	Monetary values, including unit.
QUANTITY	Measurements, as of weight or distance.
ORDINAL	"first", "second", etc.
CARDINAL	Numerals that do not fall under another type.

In [323]:
from nltk.corpus import sentiwordnet as swn
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [266]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [338]:
def get_feature_1_entities(doc, target_ent_labels=['PERSON', 'EVENT', 'ORG']):
    """
    Return 0 = no entities, 1 = one or more entities
    """
    
    ent_labels = [token.ent_type_ for token in doc]
    has_name_entity = int(bool(set(target_ent_labels) & set(ent_labels)))
    return has_name_entity

def get_feature_2_com_sup(doc, target_tags=['JJR', 'JJS', 'RBR', 'RBS']):
    """
    Return 0 = no entries, 1 = at least one adjective or adverb -  comparative/ superlative
    """
    
    pos_tags = [token.tag_ for token in doc]
    has_comparative_superlative = int(bool(set(target_tags) & set(pos_tags)))
    return has_comparative_superlative


def get_feature_3_swn_emotion(raw_sentence):
    """
    Return 0 = no emotion, 1 = emotion
    """
    
    lemmatizer = WordNetLemmatizer()
    sentiment = 0.0

    tagged_sentence = pos_tag(word_tokenize(raw_sentence))

    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue
            
        # Try to take 5 synsets 
        # Take the first synset
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())

        sentiment += swn_synset.pos_score() - swn_synset.neg_score()

    # print(sentiment)
 
    # Has an emotion
    if sentiment > 0.3 or sentiment < -0.3:
        return 1

    return 0

In [342]:
def get_feature_df(file_name='examiner-headlines.txt'):
    f_lines = read_lines_fron_txt(file_name)
    stat= []
    for line in f_lines: 
        doc = nlp(line)   
        stat.append([get_feature_1_entities(doc=doc),\
                     get_feature_2_com_sup(doc=doc),\
                     get_feature_3_swn_emotion(raw_sentence=line)])
    df = pd.DataFrame(stat, columns=['has_entity', 'has_com_sup', 'has_emotion'])
    return df

In [343]:
df = get_feature_df()

In [347]:
print('Has at least one entity of [PERSON, EVENT, ORG]: {}%'\
      .format(100*df['has_entity'].sum()/df['has_entity'].count()))

Has at least one entity of [PERSON, EVENT, ORG]: 59.56%


In [348]:
print('Has at least one adj/adv + comparative/ superlative: {}%'\
      .format(100*df['has_com_sup'].sum()/df['has_com_sup'].count()))

Has at least one adj/adv + comparative/ superlative: 4.48%


In [349]:
print('Has an emotion: {}%'\
      .format(100*df['has_emotion'].sum()/df['has_emotion'].count()))

Has an emotion: 28.44%


# Step

## Колокації

In [447]:
from nltk.corpus import wordnet as wn
import spacy
from tqdm import tqdm
nlp = spacy.load('en')

In [370]:
def getSynonyms(lemma, pos=wn.VERB):
    synsets = wn.synsets(lemma, pos=pos)
    if not synsets:
        return None
    else:
        synonyms =[]
        for syn in synsets:
            synonyms = synonyms + [l.name() for l in syn.lemmas()]
        return(synonyms)

In [382]:
lemmas = ["say", "tell", "speak", "claim", "communicate"]
synonyms = []
for lemma in lemmas:
    synonyms = synonyms + getSynonyms(lemma, pos=wn.VERB)
synonyms = [item for item in synonyms if '_' not in item]
synonyms = list(set(synonyms))
synonyms

['say',
 'differentiate',
 'arrogate',
 'transmit',
 'claim',
 'allege',
 'secern',
 'secernate',
 'severalize',
 'assure',
 'suppose',
 'distinguish',
 'read',
 'mouth',
 'communicate',
 'evidence',
 'pass',
 'state',
 'order',
 'enjoin',
 'enunciate',
 'verbalise',
 'aver',
 'recite',
 'utter',
 'tell',
 'enounce',
 'articulate',
 'severalise',
 'separate',
 'speak',
 'intercommunicate',
 'narrate',
 'recount',
 'exact',
 'pronounce',
 'verbalize',
 'convey',
 'talk',
 'take',
 'commune',
 'address']

In [445]:
def getCollocations(file_name='blog2008.txt'):
    f_lines = read_lines_fron_txt(file_name)

    verb_adv_list = []
    for line in tqdm(f_lines):
        doc = nlp(line)
        for token in doc:
            if(token.lemma_ in synonyms and token.pos_ == 'VERB'):
                for child in token.children:
                    if child.pos_ == 'ADV' and child.dep_ == 'advmod' and child.text[-2:] == 'ly':
    #                     print(token.lemma_, child.text, child.pos_, child.dep_)
                        verb_adv_list.append([token.lemma_, child.text])
    #                     print(line)
    return verb_adv_list


In [448]:
verb_adv_list = getCollocations(file_name='blog2008.txt')

100%|██████████| 303994/303994 [1:04:51<00:00, 78.12it/s]


In [ ]:
import pickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

save_obj(verb_adv_list, 'my_collocations')

In [478]:
df_cols =  pd.DataFrame(verb_adv_list, columns=['verb', 'adverb'])
df_cols = df_cols.assign(count=1)

In [479]:
df_agg = df_cols.groupby(['verb', 'adverb']).agg({'count':'sum'})

In [480]:
 g = df_agg['count'].groupby(level=0, group_keys=False)

In [483]:
 res = g.apply(lambda x: x.sort_values(ascending=False).head(10))

### Result

In [485]:
res['say']

adverb
recently      73
actually      73
repeatedly    55
simply        45
explicitly    39
publicly      36
basically     30
really        29
only          25
clearly       22
Name: count, dtype: int64

In [486]:
res['tell']

adverb
recently      25
reportedly    14
privately     10
basically      9
only           9
repeatedly     8
simply         8
really         7
probably       7
actually       7
Name: count, dtype: int64

In [487]:
res['speak'] 

adverb
directly        34
publicly        15
fiercely        12
only            10
recently         8
loudly           7
clearly          7
openly           7
actually         6
figuratively     5
Name: count, dtype: int64

In [488]:
res['claim'] 

adverb
falsely       50
Falsely       17
previously     8
repeatedly     8
actually       3
laughably      3
publicly       3
credibly       3
initially      3
recently       3
Name: count, dtype: int64

In [489]:
res["communicate"]

adverb
effectively    3
directly       3
regularly      1
quickly        1
profoundly     1
privately      1
poorly         1
loudly         1
hopefully      1
freely         1
Name: count, dtype: int64

In [493]:
res.reset_index().to_csv('collocations.csv', index=False)